# Tennis Task for Collaboration and Competition
---

### 1. Import the Necessary Packages

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from utilities import transpose_list, transpose_to_tensor
from buffer import ReplayBuffer

from maddpg_agent import MADDPG

%matplotlib inline

### 2. Instantiate the Environment and Agent

In [2]:
env = UnityEnvironment(file_name="./Tennis_Windows_x86_64/Tennis.exe")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
START_DECAY = 10000
GOAL_SCORE = .5

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


### 3. Train the Agent with DDPG

In [3]:
BUFFER_SIZE = int(5e6)  # replay buffer size
BATCH_SIZE = 128  # minibatch size
GAMMA = 0.99  # discount factor
TAU = 1e-3  # for soft update of target parameters
LR_ACTOR = 1e-4  # learning rate of the actor
LR_CRITIC = 1e-4  # learning rate of the critic
WEIGHT_DECAY = 0 # L2 penalty to the cost function to shrink weights
INITIAL_NOISE = 2 # how much OU Noise to start with
NOISE_DECAY = 0.9999 # how fast to decay OU noise
UPDATE_EVERY = 4 # how often to update the 
SOLVED = False # whether the task has been solved or not

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def main(number_of_episodes=500, num_agents = 2, print_every=100):

    scores_deque = deque(maxlen=print_every)
    scores = []
    torch.set_num_threads = num_agents
    shared_buffer = ReplayBuffer(BUFFER_SIZE)
    maddpg = MADDPG()
    agent1_reward = []
    agent2_reward = []

    # training loop
    # show progressbar
    import progressbar as pb
    widget = ['episode: ', pb.Counter(),'/',str(number_of_episodes),' ', 
              pb.Percentage(), ' ', pb.ETA(), ' ', pb.Bar(marker=pb.RotatingMarker()), ' ' ]
    timer    = pb.ProgressBar(widgets=widget, maxval=number_of_episodes).start()
    noise = INITIAL_NOISE


    for i_episode in range(1, number_of_episodes+1):
        timer.update(i_episode)
        reward_this_episode = np.zeros((num_agents, 2)) # TODO why is this number 2?
        env_info = env.reset(train_mode=False)[brain_name]
        all_observations = env_info.vector_observations
        import pdb; pdb.set_trace()
        obs, obs_full = transpose_list(all_observations)

        agent.reset()
        score = 0
        t = 0

        while True:
            t += num_agents
            actions = maddpg.act(transpose_to_tensor(obs), noise=noise)
            noise *= NOISE_DECAY

            actions_array = torch.stack(actions).detach(numpy())

            actions_for_env = np.rollaxis(actions_array, 1)

            next_obs, next_obs_full, rewards, dones, info = env.step(actions_for_env)[brain_name]

            transition = (obs, obs_full, actions_for_env, rewards, next_obs, next_obs_full, dones)

            shared_buffer.push(transition)

            reward_this_episode += rewards

            obs, obs_full = next_obs, next_obs_full

        if len(buffer) > BATCH_SIZE and episode % episode_per_update < parallel_envs:
            for a_i in range(2):
                samples = buffer.sample(batchsize)
                maddpg.update(samples, a_i, logger)
            maddpg.update_targets()

        for i in range(num_agents):
            agent0_reward.append(reward_this_episode[i, 0]) 
            agent1_reward.append(reward_this_episode[i, 1]) 
        
        if episode % 100 == 0 or episode == number_of_episodes-1:
            avg_rewards = [np.mean(agent0_reward), np.mean(agent1_reward)]
            agent0_reward = []
            agent1_reward = []
            for a_i, avg_reward in enumerate(avg_rewards):
                logger.add_scalar('agent%i/mean_episode_rewards' % a_i, avg_reward, episode)


            # actions = [agent.act(state) for state in states]
            # print(f"actions {actions}")
            # actions = np.vstack(actions)
            # env_info = env.step(actions)[brain_name]
            # next_states = env_info.vector_observations
            # rewards = env_info.rewards
            # dones = env_info.local_done
            # agent.step(states, actions, rewards, next_states, dones)
            # states = next_states
            # score += np.amax(rewards)
            # assumption, episode will terminate for both agents at the same time
            if True in dones:
                break 
        scores_deque.append(avg_rewards)
        scores.append(avg_rewards)
        print('\rEpisode {}\tAverage Scores: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        if np.amax(np.mean(scores_deque)) >= GOAL_SCORE and not SOLVED:
            SOLVED = True
            print(f'Environment solved! achieved an average score of 30 over 100 episodes at episode {i_episode}')
            #saving model
            save_dict_list =[]
            if save_info:
                for i in range(3):

                    save_dict = {'actor_params' : maddpg.maddpg_agent[i].actor.state_dict(),
                                'actor_optim_params': maddpg.maddpg_agent[i].actor_optimizer.state_dict(),
                                'critic_params' : maddpg.maddpg_agent[i].critic.state_dict(),
                                'critic_optim_params' : maddpg.maddpg_agent[i].critic_optimizer.state_dict()}
                    save_dict_list.append(save_dict)

                    torch.save(save_dict_list, 
                            os.path.join(model_dir, 'episode-{}.pt'.format(episode)))
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
                        
    return scores

scores = main()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

episode: 0/500 N/A% ETA:  --:--:-- |/                                        | 

ValueError: too many values to unpack (expected 2)

In [ ]:
env.close()

### 4. Watch a Smart Agent!

In [ ]:
def run_agent(n_episodes=1, max_t=300, print_every=100):
    score = 0
    agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
    agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=False)[brain_name]
        state = env_info.vector_observations
        agent.reset()

        while True:
            action = agent.act(state, add_noise=False, decay_noise=decay)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            state = next_state
            score += reward
            if done:
                break 
    return score

score = run_agent()

print(f"Agent Complete Episode With a Score Of: {score}")